## Overview

#### written on 09/29/24

#### Following tasks had all been accomplished.

added more columns to tables, which need to be filled.
go over the column of match_id of matches table, fetch the game data for each game.

matches (update)
- game_start (timestamp)
- game_end (timestamp)

match_account (update)
- damage_to_physical (int)
- damage_to_magic (int)
- damage_to_true (int)
- damaged_mitigated (int)
- damaged_self_healed (int)

also, for each user, update the user info.

accounts (update)
- user_name (varchar(16))
- tag (varchar(5))
- lvl (smallint)

after all,

- remove:
    - game_creation column from matches
    - heal_total from match_account

## Import

In [1]:
import time
import importlib
import api_caller
import db_connector

## API call setting

In [2]:
# Reload the module to ensure the latest changes are reflected
importlib.reload(api_caller)

caller = api_caller.APICaller()

# Test your function call
# result = caller.get_account_from_riot_id("NoMatterWhoUR", "NA1")
# print(result)

## DB connection setting

In [3]:
# Reload the module to ensure the latest changes are reflected
importlib.reload(db_connector)

connector = db_connector.DBConnector()

## Query preparation

In [4]:
# matches

'''
original query:
INSERT INTO matches (match_id)
VALUES (%s)
ON CONFLICT (match_id)
DO UPDATE SET game_start = %s, game_end = %s
'''

matches_query = {
    "INSERT": "INTO matches",
    "ON CONFLICT": "(match_id)",
    "DO UPDATE SET": ""
}

matches_col = ["match_id"]
matches_update_col = ["game_start", "game_end"]

In [5]:
# accounts

'''
original query:
INSERT INTO accounts (puuid)
VALUES (%s)
ON CONFLICT (puuid)
DO UPDATE SET user_name = %s, tag = %s, lvl = %s
'''

accounts_query = {
    "INSERT": "INTO accounts",
    "ON CONFLICT": "(puuid)",
    "DO UPDATE SET": ""
}

accounts_col = ["puuid"]
accounts_update_col = ["user_name", "tag", "lvl"]

In [6]:
# match_account

'''
original query:
INSERT INTO match_account (match_id, puuid)
VALUES (%s, %s)
ON CONFLICT (match_id, puuid)
DO UPDATE SET damage_to_physical = %s, damage_to_magic = %s,  damage_to_true = %s, damaged_mitigated = %s, damaged_self_heald
'''

match_account_query = {
    "INSERT": "INTO match_account",
    "ON CONFLICT": "(match_id, puuid)",
    "DO UPDATE SET": ""
}

match_account_col = ["match_id", "puuid"]
match_account_update_col = ["damage_to_physical", "damage_to_magic", "damage_to_true", "damaged_mitigated", "damaged_self_healed"]

## Main code

In [7]:
# filter by game_start not filled in the future
# "select match_id from matches where game_start IS NOT NULL"
query = {
    "SELECT": "match_id",
    "FROM": "matches"
}
matches = connector.select_query(query)
# call 100 matches at a time
# print(len(matches))
# print(matches[:10])

In [8]:
for idx, match in enumerate(matches):
    # print(idx)
    # each match is a tuple with a single element for match list from DB
    data = caller.get_match_from_match_id(match[0])

    # conversion to epoch second
    start_epoch = data["info"]["gameStartTimestamp"] // 1000
    end_epoch = data["info"]["gameEndTimestamp"] // 1000

    # insert (update) matches table
    matches_val = (
        data["metadata"]["matchId"],
        time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(start_epoch)),
        time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(end_epoch))
    )

    connector.insert_query(matches_query, matches_col, matches_val, matches_update_col)

    for i in range(10):

        accounts_val = (data["info"]["participants"][i]["puuid"], 
                        data["info"]["participants"][i]["riotIdGameName"],
                        data["info"]["participants"][i]["riotIdTagline"],
                        data["info"]["participants"][i]["summonerLevel"]
        )
    
        match_account_val = (data["metadata"]["matchId"],
                             data["info"]["participants"][i]["puuid"], 
                             data["info"]["participants"][i]["physicalDamageDealtToChampions"],
                             data["info"]["participants"][i]["magicDamageDealtToChampions"],
                             data["info"]["participants"][i]["trueDamageDealtToChampions"],
                             data["info"]["participants"][i]["damageSelfMitigated"],
                             # self heal = total heal - team heal
                             data["info"]["participants"][i]["totalHeal"] - data["info"]["participants"][i]["totalHealsOnTeammates"]
        )
        connector.insert_query(accounts_query, accounts_col, accounts_val, accounts_update_col)
        connector.insert_query(match_account_query, match_account_col, match_account_val, match_account_update_col)
    if (idx + 1) % 10 == 0:
        print(idx + 1, " matches have been updated")

10  matches have been updated
20  matches have been updated
30  matches have been updated
40  matches have been updated
50  matches have been updated
60  matches have been updated
70  matches have been updated
80  matches have been updated
90  matches have been updated
have called 100 apis: 2 min break starts
100  matches have been updated
110  matches have been updated
120  matches have been updated
130  matches have been updated
140  matches have been updated
150  matches have been updated
160  matches have been updated
170  matches have been updated
180  matches have been updated
190  matches have been updated
have called 200 apis: 2 min break starts
200  matches have been updated
210  matches have been updated
220  matches have been updated
230  matches have been updated
240  matches have been updated
250  matches have been updated
260  matches have been updated
270  matches have been updated
280  matches have been updated
290  matches have been updated
have called 300 apis: 2 min 

In [9]:
print("commited and ended")
connector.commit()
connector.end()

commited and ended
